In [44]:
import tempfile

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import shap
import tqdm
import trueskill
from hyperopt import STATUS_OK, SparkTrials, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope
from sklearn.compose import ColumnTransformer
from sklearn.metrics import ndcg_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from tqdm import tqdm

from JapanHorseRaceAnalytics.utilities.base import (
    get_random_seed,
    get_spark_session,
    read_hive_table,
)
from JapanHorseRaceAnalytics.utilities.metrics import kelly_criterion
from JapanHorseRaceAnalytics.utilities.plot import plot_feature_importances
from JapanHorseRaceAnalytics.utilities.structured_logger import logger

japanize_matplotlib.japanize()

In [2]:
spark = get_spark_session()

24/07/15 14:07:16 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.56 instead (on interface en0)
24/07/15 14:07:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/15 14:07:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [29]:
data = read_hive_table(
    table_name="features_20240304_v1",
    schema="jhra_curated",
    spark_session=spark,
    # use_cache=False,
    parse_dates=["meta_発走日時"],
)

rows_before = data.shape[0]
logger.info(f"Original data length: {rows_before}")

# Drop from data where cat_トラック種別 == "障害"
# Keep only horses that have 3 races
# Keep only data from 2000 onwards
data = data[
    # (data["cat_トラック種別"] != "障害")
    (~data["meta_着順"].isna())
    # & (data["meta_異常区分"] == "0")
    # & (data["num_1走前着順"].notnull())
    # & (data["num_2走前着順"].notnull())
    # & (data["num_3走前着順"].notnull())
    # & (data["meta_発走日時"] >= "2000-01-01")
]

rows_after = data.shape[0]
logger.info(
    f"Data length after filtering: {rows_after} (dropped {rows_before - rows_after} rows, {100 * (rows_before - rows_after) / rows_before:.2f}%)"
)

# Interpolate missing values for num_馬体重 (20 instances from 1999 ~ 2017)
data["num_馬体重"] = (
    data.sort_values("meta_発走日時")
    .groupby("meta_血統登録番号")["num_馬体重"]
    .transform(lambda x: x.interpolate(method="linear", limit_direction="both"))
)

data.reset_index(drop=True, inplace=True)
data.head()

{"event": "Read from parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_20240304_v1.snappy.parquet to pandas", "level": "info", "timestamp": "2024-07-15T05:31:41.092297Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
{"event": "Original data length: 1217019", "level": "info", "timestamp": "2024-07-15T05:31:41.629195Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
{"event": "Data length after filtering: 1206122 (dropped 10897 rows, 0.90%)", "level": "info", "timestamp": "2024-07-15T05:31:41.955674Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}


,meta_単勝払戻金,meta_複勝払戻金,meta_レースキー,meta_馬番,meta_血統登録番号,meta_発走日時,meta_単勝的中,meta_単勝オッズ,meta_複勝的中,meta_複勝オッズ,...,cat_3走前休養理由分類コード,num_3走前3着タイム差,cat_トラック種別,num_距離,cat_距離区分,num_入厩何日前逆数,cat_堅実な馬,cat_過去3走中1走訳あり凡走,cat_過去3走中2走好走,cat_過去3走繋がりあり
0,0,0,09025206,11,00100027,2002-12-01 12:45:00+09:00,0,38.6,0,4.4,...,None,NaN,芝,1600,マイル,1.0,False,False,False,False
1,0,0,08032303,04,00100027,2003-02-08 10:55:00+09:00,0,39.5,0,11.8,...,None,NaN,ダート,1800,中距離,1.0,False,False,False,False
2,0,0,08032801,14,00100027,2003-02-23 10:00:00+09:00,0,31.0,0,3.7,...,None,NaN,ダート,1800,中距離,1.0,False,False,False,False
3,0,0,09031403,07,00100027,2003-03-09 10:55:00+09:00,0,17.2,0,2.3,...,None,1.4,ダート,1800,中距離,1.0,False,False,False,False
4,0,0,09031701,10,00100027,2003-03-22 10:05:00+09:00,0,20.8,0,2.7,...,None,1.1,ダート,1800,中距離,1.0,False,False,False,False


In [30]:
# Assuming no draws in horse racing
env = trueskill.TrueSkill(draw_probability=0)

df_sorted = data.sort_values(by=["meta_発走日時", "meta_レースキー"])

# Create a dictionary of horse ratings
horse_ratings = {
    horse_id: env.create_rating()
    for horse_id in df_sorted["meta_血統登録番号"].unique()
}

# Placeholder for ratings at each point in time
df_sorted["race_quality"] = pd.NA
df_sorted["rating_post_race"] = pd.NA
df_sorted["rating_uncertainty_post_race"] = pd.NA

# Iterate through races in chronological order
for (_, race_id), race_data in tqdm(df_sorted.groupby(["meta_発走日時", "meta_レースキー"])):
    race_results = race_data.sort_values("meta_着順")
    horse_ids = race_results["meta_血統登録番号"].tolist()

    # Measure race quality: 0 means significant disparity in skill levels, 1 means very close match
    horse_groups = [[horse_ratings[horse_id]] for horse_id in horse_ids]
    df_sorted.loc[race_results.index, "race_quality"] = env.quality(horse_groups)

    # Lower rank number means a better position
    ranks = list(range(len(horse_groups)))

    # Update ratings based on the race outcome
    updated_ratings = env.rate(horse_groups, ranks=ranks)

    # Apply updates to the horse ratings and DataFrame
    for index, (horse_id, new_rating_group) in zip(race_results.index, zip(horse_ids, updated_ratings)):
        horse_ratings[horse_id] = new_rating_group[0]
        df_sorted.at[index, "rating_post_race"] = horse_ratings[horse_id].mu
        df_sorted.at[index, "rating_uncertainty_post_race"] = horse_ratings[horse_id].sigma

100%|██████████| 86237/86237 [1:07:16<00:00, 21.36it/s]   


In [31]:
# Lag the rating_post_race by one row per horse (meta_血統登録番号) chronologically (meta_発走日時)
df_sorted["rating_pre_race"] = (
    df_sorted.sort_values("meta_発走日時")
    .groupby("meta_血統登録番号")["rating_post_race"]
    .shift(1)
    .fillna(env.mu)
)

df_sorted["rating_uncertainty_pre_race"] = (
    df_sorted.sort_values("meta_発走日時")
    .groupby("meta_血統登録番号")["rating_uncertainty_post_race"]
    .shift(1)
    .fillna(env.sigma)
)

In [54]:
# df_sorted.to_parquet(
#     "data/sql_tables/features_20240304_v1_with_overall_ratings.snappy.parquet",
#     index=False,
#     compression="snappy",
# )
df_sorted = pd.read_parquet("data/sql_tables/features_20240304_v1_with_overall_ratings.snappy.parquet")

In [55]:
race_rating_sum = df_sorted.groupby("meta_レースキー")["rating_pre_race"].transform("sum")
race_horse_count = df_sorted.groupby("meta_レースキー")["rating_pre_race"].transform("count")
df_sorted["mean_competitor_rating_pre_race"] = (race_rating_sum - df_sorted["rating_pre_race"]) / (race_horse_count - 1)
df_sorted["mean_competitor_rating_pre_race_diff"] = df_sorted["rating_pre_race"] - df_sorted["mean_competitor_rating_pre_race"]

race_rating_uncertainty_sum = df_sorted.groupby("meta_レースキー")["rating_uncertainty_pre_race"].transform("sum")
race_horse_count = df_sorted.groupby("meta_レースキー")["rating_uncertainty_pre_race"].transform("count")
df_sorted["mean_competitor_rating_uncertainty_pre_race"] = (race_rating_uncertainty_sum - df_sorted["rating_uncertainty_pre_race"]) / (race_horse_count - 1)
df_sorted["mean_competitor_rating_uncertainty_pre_race_diff"] = df_sorted["rating_uncertainty_pre_race"] - df_sorted["mean_competitor_rating_uncertainty_pre_race"]

In [56]:
# Calculate the average 1走前経過日数 for each horse's competitors in the same race, excluding the horse itself. Store in "mean_competitor_1走前経過日数" column.
race_sums = df_sorted.groupby('meta_レースキー')['num_1走前経過日数'].transform('sum')
race_counts = df_sorted.groupby('meta_レースキー')['num_1走前経過日数'].transform('count')
# Calculate the average for competitors of each horse
df_sorted['mean_competitor_1走前経過日数'] = (race_sums - df_sorted['num_1走前経過日数']) / (race_counts - 1)

In [57]:
data_filtered = data[
    (data["cat_トラック種別"] != "障害")
    & (~data["meta_着順"].isna())
    & (data["meta_異常区分"] == "0")
    & (data["num_1走前着順"].notnull())
    & (data["num_2走前着順"].notnull())
    & (data["num_3走前着順"].notnull())
    & (data["meta_発走日時"] >= "2000-01-01")
]

remain_horses_per_race = data_filtered.groupby("meta_レースキー")["meta_血統登録番号"].count().reset_index().set_index("meta_レースキー")
actual_horses_per_race = data_filtered[["meta_レースキー", "num_頭数"]].drop_duplicates().set_index("meta_レースキー")

# Make sure we aren't creating any duplicate rows
assert len(remain_horses_per_race) == len(actual_horses_per_race) == len(remain_horses_per_race.join(actual_horses_per_race))

combined_df = remain_horses_per_race.join(actual_horses_per_race)

target_race_ids = combined_df[combined_df["meta_血統登録番号"] == combined_df["num_頭数"]].index.unique().tolist()

df_final = df_sorted[df_sorted["meta_レースキー"].isin(target_race_ids)]

len(combined_df), len(target_race_ids), len(df_final)

(71593, 24104, 335472)

In [112]:
df = df_final.copy()

# Initialize GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=get_random_seed())

# Assume df['group'] is your grouping variable
# Splitting the data
for train_idx, test_idx in gss.split(df, groups=df["meta_レースキー"]):
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

# Now, train_df and test_df contain your split data, respecting group boundaries
X_train = train_df
y_train = train_df["meta_着順"].apply(lambda x: int(1.0 / x * 10) if x < 4 else 0)

group_counts_train = X_train.groupby('meta_レースキー').size().reset_index(name='レースキー_count')
first_occurrences_train = X_train[['meta_レースキー']].drop_duplicates().reset_index(drop=True)
merged_train = first_occurrences_train.merge(group_counts_train, on='meta_レースキー', how='left')
groups_train = merged_train["レースキー_count"].values

X_test = test_df
y_test = test_df["meta_着順"].apply(lambda x: int(1.0 / x * 10) if x < 4 else 0)

group_counts_test = X_test.groupby('meta_レースキー').size().reset_index(name='レースキー_count')
first_occurrences_train = X_test[['meta_レースキー']].drop_duplicates().reset_index(drop=True)
merged_test = first_occurrences_train.merge(group_counts_test, on='meta_レースキー', how='left')
groups_test = merged_test["レースキー_count"].values

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"groups_train: {groups_train.shape}")
print(f"groups_test: {groups_test.shape}")

X_train: (268549, 86)
X_test: (66923, 86)
y_train: (268549,)
y_test: (66923,)
groups_train: (19283,)
groups_test: (4821,)


In [113]:
def calculate_average_ndcg(model, valid_dataset) -> float:
    y_pred = model.predict(valid_dataset.data)

    ## Initialize an empty list to store the NDCG scores of each group
    ndcg_scores = []
    # The starting index of the first item in the current group
    start_idx = 0
    for group_size in valid_dataset.group:
        end_idx = start_idx + group_size
        # Slice the true labels and predictions according to the current group
        true_labels = valid_dataset.label[start_idx:end_idx]
        predictions = y_pred[start_idx:end_idx]

        # Calculate the NDCG score for the current group.
        # ndcg_score expects 2D arrays, so we add an extra dimension
        ndcg = ndcg_score([true_labels], [predictions])
        ndcg_scores.append(ndcg)

        # Update the start index for the next group
        start_idx = end_idx

    # Calculate the average NDCG score across all groups
    return np.mean(ndcg_scores)


def predict_wide(model, valid_dataset) -> float:
    """
    Predict wide and return 的中率
    """
    y_pred = model.predict(valid_dataset.data)

    ## Initialize an empty list to store the NDCG scores of each group
    # The starting index of the first item in the current group
    start_idx = 0
    for group_size in valid_dataset.group:
        end_idx = start_idx + group_size
        # Slice the true labels and predictions according to the current group
        true_labels = valid_dataset.label[start_idx:end_idx]
        predictions = y_pred[start_idx:end_idx]

        print(true_labels)
        print(predictions)

        # Calculate the NDCG score for the current group.
        # ndcg_score expects 2D arrays, so we add an extra dimension
        # ndcg = ndcg_score([true_labels], [predictions])
        # ndcg_scores.append(ndcg)

        # Update the start index for the next group
        start_idx = end_idx

    return 0.5


def predict_win(model, valid_dataset) -> float:
    return 0.5


def predict_place(model, valid_dataset) -> float:
    return 0.5


def create_objective_fn(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    groups_train: np.ndarray,
    groups_test: np.ndarray,
    experiment_name: str,
):
    def objective(params):
        mlflow.set_experiment(experiment_name=experiment_name)
        with mlflow.start_run():
            preprocessor = ColumnTransformer(
                transformers=[
                    (
                        "num",
                        StandardScaler(),
                        [
                            "num_1走前経過日数",
                            "mean_competitor_rating_pre_race_diff",
                            "mean_competitor_1走前経過日数",
                            "num_平均馬体重差",
                        ],
                    ),
                ],
                remainder="drop",
                verbose_feature_names_out=False,
            )

            X_train_prep = preprocessor.fit_transform(X_train)
            X_test_prep = preprocessor.transform(X_test)

            feature_names = preprocessor.get_feature_names_out().tolist()

            train_dataset = lgb.Dataset(
                X_train_prep,
                label=y_train,
                group=groups_train,
                feature_name=feature_names,
            )
            valid_dataset = lgb.Dataset(
                data=X_test_prep,
                label=y_test,
                group=groups_test,
                reference=train_dataset,
                free_raw_data=False,
                feature_name=feature_names,
            )
            model = lgb.train(
                params,
                train_dataset,
                valid_sets=[valid_dataset],
                num_boost_round=1000,
                callbacks=[
                    lgb.early_stopping(stopping_rounds=50),
                ],
            )

            # 1 means perfect ranking, 0 means worst ranking
            average_ndcg = calculate_average_ndcg(model, valid_dataset)
            loss = -average_ndcg

            mlflow.log_params(params)
            mlflow.log_metric("loss", loss)
            mlflow.log_metric("average_ndcg", average_ndcg)

            wide_hit_rate = predict_wide(model, valid_dataset)
            win_hit_rate = predict_win(model, valid_dataset)
            place_hit_rate = predict_place(model, valid_dataset)

            mlflow.log_metrics({
                "wide_hit_rate": wide_hit_rate,
                "win_hit_rate": win_hit_rate,
                "place_hit_rate": place_hit_rate,
            })

            # Tree Plot
            ax = lgb.plot_tree(
                model,
                figsize=(20, 20),
                show_info=["split_gain", "data_percentage"],
                precision=2,
            )
            fig = ax.get_figure()
            with tempfile.NamedTemporaryFile(prefix="lgbm_tree_", suffix=".png") as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Plot (Gain)
            fig, ax = plot_feature_importances(
                feature_names=preprocessor.get_feature_names_out(),
                feature_importances=model.feature_importance(importance_type="gain"),
                top_n=50,
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_gain_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Data (Gain)
            feature_importances = zip(
                preprocessor.get_feature_names_out(),
                model.feature_importance(importance_type="gain"),
            )
            feature_importances_df = (
                pd.DataFrame(feature_importances, columns=["feature", "importance"])
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_gain_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # Feature Importances Plot (Split)
            fig, ax = plot_feature_importances(
                feature_names=preprocessor.get_feature_names_out(),
                feature_importances=model.feature_importance(importance_type="split"),
                top_n=50,
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_split_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Data (Split)
            feature_importances = zip(
                preprocessor.get_feature_names_out(),
                model.feature_importance(importance_type="split"),
            )
            feature_importances_df = (
                pd.DataFrame(feature_importances, columns=["feature", "importance"])
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_split_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # SHAP values
            X_test_sample = X_test.sample(n=5000, random_state=get_random_seed())
            X_test_sample_prep = preprocessor.transform(X_test_sample)
            explainer = shap.TreeExplainer(
                model=model,
                feature_names=preprocessor.get_feature_names_out(),
            )
            shap_values = explainer(X_test_sample_prep)

            # Takes a long time
            # shap_interaction_values = explainer.shap_interaction_values(X_test_sample_prep)

            # SHAP beeswarm plot
            shap.plots.beeswarm(shap_values, show=False, max_display=25)
            plt.tight_layout()
            with tempfile.NamedTemporaryFile(
                prefix="shap_beeswarm_", suffix=".png"
            ) as f:
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # SHAP interaction values heatmap
            # fig, ax = plot_shap_interaction_values(
            #     shap_interaction_values, preprocessor.get_feature_names_out()
            # )
            # with tempfile.NamedTemporaryFile(
            #     prefix="shap_interactions_", suffix=".png"
            # ) as f:
            #     fig.savefig(f.name)
            #     plt.close()
            #     mlflow.log_artifact(f.name)

            # SHAP bar plot
            shap.plots.bar(shap_values, show=False, max_display=25)
            plt.tight_layout()
            with tempfile.NamedTemporaryFile(prefix="shap_bar_", suffix=".png") as f:
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            return {"loss": loss, "params": params, "status": STATUS_OK}

    return objective

In [114]:
space = {
    "num_leaves": scope.int(hp.quniform("num_leaves", 20, 150, 1)),
    "learning_rate": hp.loguniform("learning_rate", -5, 0),  # between e^-5 and 1
    "min_data_in_leaf": hp.choice("min_data_in_leaf", range(20, 100)),
    "feature_fraction": hp.uniform("feature_fraction", 0.5, 1.0),
    "bagging_fraction": hp.uniform("bagging_fraction", 0.5, 1.0),
    "bagging_freq": hp.choice("bagging_freq", range(1, 10)),
    "lambda_l1": hp.uniform("lambda_l1", 0, 10),
    "lambda_l2": hp.uniform("lambda_l2", 0, 10),
    "max_depth": hp.choice("max_depth", range(3, 15)),
    # Constant parameters
    "objective": "lambdarank",
    "metric": ["ndcg", "map"],
    "ndcg_eval_at": [1, 3, 5],
    "verbose": -1,
    "seed": get_random_seed(),
    "boosting_type": "gbdt",
}

In [115]:
experiment_name = "20240715-simple1"

if mlflow.get_experiment_by_name(experiment_name) is None:
    mlflow.create_experiment(experiment_name)

fn = create_objective_fn(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    groups_train=groups_train,
    groups_test=groups_test,
    experiment_name=experiment_name,
)

In [116]:
trials = Trials()
fmin(
    fn=fn,
    space=space,
    algo=tpe.suggest,
    max_evals=1,
    trials=trials,
)

# trials = SparkTrials(parallelism=3, spark_session=spark)
# fmin(fn=fn, space=space, algo=tpe.suggest, max_evals=30, trials=trials)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

build_posterior_wrapper took 0.003480 seconds
TPE using 0 trials


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:                   
[34]	valid_0's ndcg@1: 0.218911	valid_0's ndcg@3: 0.371441	valid_0's ndcg@5: 0.442702	valid_0's map@1: 0.485169	valid_0's map@3: 0.323181	valid_0's map@5: 0.411111
[ 0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  5.  3.  0.  0.  0.]
[ 0.08646785  0.1183198  -0.01052079 -0.1099875  -0.16253873 -0.07680031
 -0.11077057 -0.36475272 -0.1682967   0.06222098  0.22868444  0.09316164
 -0.21422012  0.05106875 -0.26447351 -0.18686774]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  5.  0.  3.  0. 10.  0.]
[ 0.06333558 -0.19203184 -0.13488421  0.01380669 -0.17759961 -0.19941526
  0.02112122 -0.21449222 -0.19458782 -0.43301729  0.015047    0.15725634
 -0.19536015 -0.1839757   0.37690704  0.15779285]
[ 0.  0.  0. 10.  5.  0.  0.  0.  3.  0.  0.  0.]    
[-0.01047821  0.32709451  0.1459729  -0.14615954 -0.01566738 -0.12868563
 -0.38668684  0.03107495 -0.01916204  0.05747127 -0.34204913 -0.023358

KeyboardInterrupt: 